# Getting The Data

In [1]:
import pandas as pd
ghi_df = pd.read_csv('/content/drive/MyDrive/predicted_ghi.csv')
print(ghi_df)

    hour  predicted_ghi        date
0      0       0.000000  2025-10-09
1      1       0.000000  2025-10-09
2      2       0.000000  2025-10-09
3      3       0.000000  2025-10-09
4      4       0.000000  2025-10-09
5      5       0.000000  2025-10-09
6      6       0.000000  2025-10-09
7      7      65.597340  2025-10-09
8      8     174.463470  2025-10-09
9      9     298.220580  2025-10-09
10    10     443.440770  2025-10-09
11    11     560.139600  2025-10-09
12    12     639.329400  2025-10-09
13    13     648.836400  2025-10-09
14    14     606.947630  2025-10-09
15    15     491.622770  2025-10-09
16    16     326.187070  2025-10-09
17    17     157.321370  2025-10-09
18    18      38.067554  2025-10-09
19    19       0.000000  2025-10-09
20    20       0.000000  2025-10-09
21    21       0.000000  2025-10-09
22    22       0.000000  2025-10-09
23    23       0.000000  2025-10-09


In [2]:
import pandas as pd
load_df = pd.read_csv('/content/drive/MyDrive/predicted_load.csv')
print(load_df)

    hour  predicted_load        date
0      0       7685.0930  2025-10-09
1      1       7156.7070  2025-10-09
2      2       6732.7060  2025-10-09
3      3       6442.0140  2025-10-09
4      4       6328.8784  2025-10-09
5      5       6341.0220  2025-10-09
6      6       6510.8657  2025-10-09
7      7       6838.9023  2025-10-09
8      8       7046.9490  2025-10-09
9      9       7328.2065  2025-10-09
10    10       7572.7905  2025-10-09
11    11       7843.7420  2025-10-09
12    12       8167.2134  2025-10-09
13    13       8484.8960  2025-10-09
14    14       8830.7280  2025-10-09
15    15       9254.6880  2025-10-09
16    16       9659.0670  2025-10-09
17    17      10019.4470  2025-10-09
18    18      10153.8430  2025-10-09
19    19       9991.4900  2025-10-09
20    20       9671.2020  2025-10-09
21    21       9373.9290  2025-10-09
22    22       8919.2650  2025-10-09
23    23       8334.5970  2025-10-09


# To Find PV and Battery Sizes

In [18]:
battery_capacity_list = []
final_PV_plant_m2_list = []

def simulate_day(target_date, seasonal_load_df, seasonal_ghi_df):

  load_day = seasonal_load_df[seasonal_load_df['date'] == target_date].reset_index(drop=True)
  ghi_day = seasonal_ghi_df[seasonal_ghi_df['date'] == target_date].reset_index(drop=True)

  '''Finding the Final Solar Photovoltaic Plant in m²'''
  PV_plant_sizes_m2 = []
  for hour in range(24):

    load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])
    PV_with_bonus = 0.20 * 1.31
    PR = 0.786

    if hour_GHI_Wh_m2 > 0:
      plant_size_m2 = (load_for_the_hour_Wh) / (hour_GHI_Wh_m2 * PV_with_bonus * PR)
      PV_plant_sizes_m2.append(plant_size_m2)

  final_PV_plant_m2 = 0
  for x in PV_plant_sizes_m2:
    final_PV_plant_m2 += x
  final_PV_plant_m2 = final_PV_plant_m2 // (len(PV_plant_sizes_m2))
  final_PV_plant_m2_list.append(final_PV_plant_m2)

  print(f'Solar PV Plant Size {final_PV_plant_m2:} m²\n')


  '''Hourly Solar Photovoltaic Output & Surplus/Shortage Calculations'''
  total_days_shortage = 0
  total_days_surplus = 0

  battery_lowest = 0
  lowest_values = []

  for hour in range(24):
    load_for_the_hour_Wh = (load_day.iloc[hour,1]) * 1000000
    hour_GHI_Wh_m2 = (ghi_day.iloc[hour,1])

    if hour_GHI_Wh_m2 > 0:
      hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_m2) / 1000000
      diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)

      if diff >= 0:
        total_days_surplus += diff
      else:
        total_days_shortage += abs(diff)

      print(
          f"{hour}: Solar PV output: {hour_PV_output_MWh} MWh & "
          f"Actual load: {load_for_the_hour_Wh / 1000000} MWh & "
          f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
      )

      battery_lowest += diff
      lowest_values.append(battery_lowest)

    else:
      print(f'{hour}: Shortage of -{load_for_the_hour_Wh / 1000000} MWh') # shortage
      total_days_shortage += load_for_the_hour_Wh / 1000000
      battery_lowest +=  -load_for_the_hour_Wh / 1000000
      lowest_values.append(battery_lowest)

  print(f'\nSurplus: {total_days_surplus} MWh in Solar PV going to Battery Storage')
  print(f'Shortage: {total_days_shortage} MWh either accessed from Battery Storage or NGCC')
  print(lowest_values)

  '''Capacity of Battery'''
  battery_capacity = abs(min(lowest_values))
  battery_capacity_list.append(battery_capacity)

In [19]:
''' Load Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
seasonal_file = os.path.join(save_directory, 'seasonal_predicted_loads.csv')

# Check if the file exists and is non-empty
if os.path.exists(seasonal_file) and os.path.getsize(seasonal_file) > 0:
    try:
        seasonal_load_df = pd.read_csv(seasonal_file)
        print("✅ Seasonal data loaded successfully!")
        print(seasonal_load_df)
    except pd.errors.EmptyDataError:
        print("⚠️ Seasonal file exists but is empty.")
        seasonal_load_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ Seasonal file does not exist or is empty.")
    seasonal_load_df = pd.DataFrame()  # create empty DataFrame


✅ Seasonal data loaded successfully!
     hour  predicted_load        date
0       0     6730.208000  2025-03-21
1       1     6317.309600  2025-03-21
2       2     5992.536000  2025-03-21
3       3     5779.637700  2025-03-21
4       4     5609.925300  2025-03-21
..    ...             ...         ...
283    19     8018.749023  2025-02-21
284    20     8250.614258  2025-02-21
285    21     8259.625977  2025-02-21
286    22     8042.062500  2025-02-21
287    23     7776.392090  2025-02-21

[288 rows x 3 columns]


In [20]:
''' Solar GHI Data for Representative Dates'''

import pandas as pd
import os

# Path to your seasonal file
save_directory = '/content/drive/MyDrive'
GHI_file = os.path.join(save_directory, 'seasonal_predicted_ghi.csv')

# Check if the file exists and is non-empty
if os.path.exists(GHI_file) and os.path.getsize(GHI_file) > 0:
    try:
        seasonal_ghi_df = pd.read_csv(GHI_file)
        print("✅ GHI data loaded successfully!")
        print(seasonal_ghi_df)
    except pd.errors.EmptyDataError:
        print("⚠️ GHI file exists but is empty.")
        seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame
else:
    print("⚠️ GHI file does not exist or is empty.")
    seasonal_ghi_df = pd.DataFrame()  # create empty DataFrame


✅ GHI data loaded successfully!
     hour  predicted_ghi        date
0       0            0.0  2025-03-21
1       1            0.0  2025-03-21
2       2            0.0  2025-03-21
3       3            0.0  2025-03-21
4       4            0.0  2025-03-21
..    ...            ...         ...
283    19            0.0  2025-02-21
284    20            0.0  2025-02-21
285    21            0.0  2025-02-21
286    22            0.0  2025-02-21
287    23            0.0  2025-02-21

[288 rows x 3 columns]


In [21]:
simulate_day("2025-03-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-04-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-05-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 411270898.0 m²

0: Shortage of -6730.208 MWh
1: Shortage of -6317.3096 MWh
2: Shortage of -5992.536 MWh
3: Shortage of -5779.6377 MWh
4: Shortage of -5609.9253 MWh
5: Shortage of -5574.014 MWh
6: Shortage of -5655.4165 MWh
7: Solar PV output: 1373.6233480371775 MWh & Actual load: 5845.335 MWh & Shortage of -4471.7116519628225 MWh
8: Solar PV output: 10683.439055003944 MWh & Actual load: 6086.4697 MWh & Surplus of 4596.969355003945 MWh
9: Solar PV output: 17985.153265403485 MWh & Actual load: 6405.8887 MWh & Surplus of 11579.264565403486 MWh
10: Solar PV output: 26185.281121342985 MWh & Actual load: 6707.161 MWh & Surplus of 19478.120121342985 MWh
11: Solar PV output: 32821.852885704 MWh & Actual load: 6735.9727 MWh & Surplus of 26085.880185704 MWh
12: Solar PV output: 43955.41469911763 MWh & Actual load: 6783.007 MWh & Surplus of 37172.40769911763 MWh
13: Solar PV output: 47682.98779500023 MWh & Actual load: 6813.663 MWh & Surplus of 40869.324795000226 MWh
14: Solar

In [22]:
simulate_day("2025-06-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-07-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-08-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 431492642.0 m²

0: Shortage of -10040.7861328125 MWh
1: Shortage of -9209.10546875 MWh
2: Shortage of -8607.013671875 MWh
3: Shortage of -8225.138671875 MWh
4: Shortage of -7970.302734375 MWh
5: Shortage of -7622.4267578125 MWh
6: Shortage of -7478.98486328125 MWh
7: Solar PV output: 7647.28543930694 MWh & Actual load: 7438.9345703125 MWh & Surplus of 208.35086899444013 MWh
8: Solar PV output: 21032.39399833336 MWh & Actual load: 7602.40087890625 MWh & Surplus of 13429.993119427108 MWh
9: Solar PV output: 35593.448609854924 MWh & Actual load: 8006.681640625 MWh & Surplus of 27586.766969229924 MWh
10: Solar PV output: 54810.38055944139 MWh & Actual load: 8403.2412109375 MWh & Surplus of 46407.13934850389 MWh
11: Solar PV output: 67869.44158924921 MWh & Actual load: 9075.12109375 MWh & Surplus of 58794.32049549921 MWh
12: Solar PV output: 76347.77966933779 MWh & Actual load: 9823.2373046875 MWh & Surplus of 66524.54236465029 MWh
13: Solar PV output: 78561.03837402903 

In [23]:
simulate_day("2025-09-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-10-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-11-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 12575188911.0 m²

0: Shortage of -8827.7314453125 MWh
1: Shortage of -8146.84375 MWh
2: Shortage of -7746.16064453125 MWh
3: Shortage of -7347.4169921875 MWh
4: Shortage of -7165.83984375 MWh
5: Shortage of -7210.74951171875 MWh
6: Shortage of -7434.9755859375 MWh
7: Solar PV output: 58270.5786690145 MWh & Actual load: 7739.96875 MWh & Surplus of 50530.6099190145 MWh
8: Solar PV output: 380939.33675635036 MWh & Actual load: 7886.7626953125 MWh & Surplus of 373052.57406103786 MWh
9: Solar PV output: 833443.6240897849 MWh & Actual load: 8241.267578125 MWh & Surplus of 825202.3565116599 MWh
10: Solar PV output: 1216792.2496679362 MWh & Actual load: 8642.4580078125 MWh & Surplus of 1208149.7916601237 MWh
11: Solar PV output: 1824577.5171264494 MWh & Actual load: 9059.1083984375 MWh & Surplus of 1815518.408728012 MWh
12: Solar PV output: 2098199.1035388093 MWh & Actual load: 9587.666015625 MWh & Surplus of 2088611.4375231843 MWh
13: Solar PV output: 2082058.148708244 MWh

In [24]:
simulate_day("2025-12-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-01-21", seasonal_load_df, seasonal_ghi_df)
simulate_day("2025-02-21", seasonal_load_df, seasonal_ghi_df)

# print("--")
# print(battery_capacity_list)
# print(final_PV_plant_m2_list)

Solar PV Plant Size 589200782.0 m²

0: Shortage of -6590.6796875 MWh
1: Shortage of -6419.68359375 MWh
2: Shortage of -6270.0166015625 MWh
3: Shortage of -6161.58837890625 MWh
4: Shortage of -6253.07177734375 MWh
5: Shortage of -6415.90869140625 MWh
6: Shortage of -6880.91845703125 MWh
7: Solar PV output: 1467.1693833617996 MWh & Actual load: 7534.10791015625 MWh & Shortage of -6066.938526794451 MWh
8: Solar PV output: 13221.187085937918 MWh & Actual load: 7992.43896484375 MWh & Surplus of 5228.748121094168 MWh
9: Solar PV output: 25029.577806753772 MWh & Actual load: 7997.8076171875 MWh & Surplus of 17031.770189566272 MWh
10: Solar PV output: 35383.0206629472 MWh & Actual load: 7715.7314453125 MWh & Surplus of 27667.2892176347 MWh
11: Solar PV output: 45267.59265580917 MWh & Actual load: 7430.4833984375 MWh & Surplus of 37837.10925737167 MWh
12: Solar PV output: 58547.42007396805 MWh & Actual load: 7292.4482421875 MWh & Surplus of 51254.97183178055 MWh
13: Solar PV output: 58203.93543

In [25]:
final_battery_size = sum(battery_capacity_list) / len(battery_capacity_list)
final_PV_plant_size = sum(final_PV_plant_m2_list) / len(final_PV_plant_m2_list)

print(f"Final Averaged Battery Capacity: {final_battery_size}")
print(f"Final Averaged PV Plant Size: {final_PV_plant_size}")

Final Averaged Battery Capacity: 57706.92400050562
Final Averaged PV Plant Size: 1304564487.4166667


# Calculating TODAY'S Information

In [30]:
'''Today's Hourly Solar PV Output & Surplus/Shortage Calculations'''

amt_in_battery = 0

for hour in range(24):
  load_for_the_hour_Wh = (load_df.iloc[hour,1]) * 1000000
  hour_GHI_Wh_m2 = (ghi_df.iloc[hour,1])

  PV_with_bonus = 0.20 * 1.31
  PR = 0.786

  if hour_GHI_Wh_m2 > 0:
    hour_PV_output_MWh = (hour_GHI_Wh_m2 * PV_with_bonus * PR * final_PV_plant_size) / 1000000
    diff = hour_PV_output_MWh - (load_for_the_hour_Wh / 1000000)
    print(
        f"{hour}: The Solar PV output: {hour_PV_output_MWh} MWh, "
        f"Actual load: {load_for_the_hour_Wh / 1000000} MWh, "
        f"{'Surplus of ' if diff >= 0 else 'Shortage of -'}{abs(diff)} MWh"
    )
    if diff >= 0:
      amt_in_battery += (diff)

  else:
    print(f'{hour}: Shortage of -{load_for_the_hour_Wh / 1000000} MWh')

print(amt_in_battery)
if amt_in_battery < final_battery_size:
  print('GOT SPACE IN BATTERY')
else:
  print('NO SPACE IN BATTERY')

0: Shortage of -7685.093 MWh
1: Shortage of -7156.707 MWh
2: Shortage of -6732.706 MWh
3: Shortage of -6442.014 MWh
4: Shortage of -6328.8784 MWh
5: Shortage of -6341.022 MWh
6: Shortage of -6510.8657 MWh
7: The Solar PV output: 17622.8286427015 MWh, Actual load: 6838.9023 MWh, Surplus of 10783.926342701501 MWh
8: The Solar PV output: 46869.88582496019 MWh, Actual load: 7046.949 MWh, Surplus of 39822.93682496019 MWh
9: The Solar PV output: 80117.42822295926 MWh, Actual load: 7328.2065 MWh, Surplus of 72789.22172295926 MWh
10: The Solar PV output: 119131.06084633322 MWh, Actual load: 7572.7905 MWh, Surplus of 111558.27034633321 MWh
11: The Solar PV output: 150482.38521243943 MWh, Actual load: 7843.742 MWh, Surplus of 142638.64321243943 MWh
12: The Solar PV output: 171756.84962898135 MWh, Actual load: 8167.2134 MWh, Surplus of 163589.63622898134 MWh
13: The Solar PV output: 174310.92014321504 MWh, Actual load: 8484.896 MWh, Surplus of 165826.02414321504 MWh
14: The Solar PV output: 16305

# STEPS...

1) *Pick representative dates for each season (or better: pick a small set of 3–5 representative days per season).*
2) *For each chosen date (or each day in a small seasonal sample) compute the hourly diff = PV − load, the cumulative running tally, and the battery capacity = abs(min(running_tally)).*
3) Apply the 0.0001 convergence idea: instead of adding one hour at a time (your professor’s suggestion), it’s more meaningful with seasonal sampling to add days (or if you prefer, add hours across many days). For each season:

* Start with 1 day, compute the average of the battery requirements (or average
of absolute diffs), then add one more day from the same season, recompute the average, and stop when relative change < 0.0001.
* This tells you the number of days needed from that season to reach a stable average.

^^By Friday

---

Combine seasons into a final design. Options:

* Conservative: choose the maximum battery size across seasons (safe but costly).

* Weighted: weight each season by the number of days it represents (e.g., 90 days per season) and compute a weighted average battery requirement.

* Risk-based: pick the 95th percentile battery size across all sampled days (trades cost vs. reliability).

^^By Sunday


**Representative Dates**

Spring: March 21, **April 21**, May 21

Summer: June 21, **July 21**, August 21

Fall: September 21, **October 21**, November 21

Winter: December 21, **January 21**, February 21